Playing ATARI-ram games using python-neat

# Import

In [24]:
import numpy as np
import gym
from gym import wrappers
from __future__ import print_function
import os
%matplotlib inline
import neat
import visualize
os.environ['DISPLAY']=':0'

# Define config

In [25]:
fc_config_filename = 'fc.config'
game = 'MsPacman-ram-v0'
num_evaluations = 3
num_cores = 10
population_size = 60

# Create environment

In [26]:
env = gym.make(game)

[2017-02-07 16:43:07,779] Making new env: MsPacman-ram-v0


# Create neat-python population

In [27]:
# Load configuration.
config_initial = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation, fc_config_filename)

config_initial.genome_config.num_inputs = env.observation_space.shape[0]
config_initial.genome_config.num_outputs = env.action_space.n
config_initial.pop_size = population_size

In [28]:
game_fc_config_filename = 'fc-' + game + '.config'

config_initial.save(game_fc_config_filename)

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation, game_fc_config_filename)

# Create the population, which is the top-level object for a NEAT run.
p = neat.Population(config)

# Add reporters

In [29]:
# Add a stdout reporter to show progress in the terminal.
p.add_reporter(neat.StdOutReporter())
stats = neat.StatisticsReporter()
p.add_reporter(stats)
p.add_reporter(neat.Checkpointer(5))

# Define fitness via game score

In [30]:
def transform_observation(observation):
    observation = observation / 255. * 2 - 1
    return observation

# a = argmax_a Q(s,a)
def predict_action(observation, network):
    observation = transform_observation(observation)
    output = network.activate(observation)
    action = np.argmax(output)
    return(action)

# play num_evaluations games, take mean
def evaluate_network(env, network):
    rewards = []
    i = 0
    while i < num_evaluations:
        rewards += [get_reward(env, network)]
        i += 1

    res = np.array(rewards).mean()
    return res

# play 1 game with network
def get_reward(env, network):
    observation = env.reset()
    done = False
    iteration, total_reward = 0, 0
        
    while not done:
        #env.render()
        action = predict_action(observation, network)
        observation, reward, done, info = env.step(action)
        total_reward += reward

        #if iteration % 500 == 0:
        #    print(str(iteration))

        #if total_reward < -12000 or iteration >= 8000:
        #    break

        iteration += 1
        
    return total_reward

def evaluate_genome(genome, config):
    network = neat.nn.FeedForwardNetwork.create(genome, config)
    fitness = evaluate_network(env, network)
    return fitness
    
#print(evaluate_genome(p.species.get_species(1).members[1], config))

evaluator = neat.parallel.ParallelEvaluator(num_workers = num_cores, eval_function = evaluate_genome, timeout = None)

# Run evolution

In [32]:
# Run evolution
winner = p.run(evaluator.evaluate, 100)


 ****** Running generation 1 ****** 

Population's average fitness: 266.66667 stdev: 223.56290
Best fitness: 1430.00000 - size: (9, 1141) - species 1 - id 107
Species length: 1 totaling 60 individuals
Species no improv: {1: 0}
Average adjusted fitness: 0.151
Spawn amounts: [60]
Species fitness  : [0.15085158150851588]
Mean genetic distance 1.20601267575, std dev 0.126455031622
Total extinctions: 0
Generation time: 74.640 sec (72.038 average)

 ****** Running generation 2 ****** 

Population's average fitness: 281.77778 stdev: 157.91051
Best fitness: 666.66667 - size: (9, 1141) - species 1 - id 129
Species length: 1 totaling 60 individuals
Species no improv: {1: 1}
Average adjusted fitness: 0.366
Spawn amounts: [60]
Species fitness  : [0.3655677655677656]
Mean genetic distance 1.49864700267, std dev 0.173989511475
Total extinctions: 0
Generation time: 78.576 sec (74.217 average)

 ****** Running generation 3 ****** 

Population's average fitness: 263.44444 stdev: 194.87125
Best fitness

CompleteExtinctionException: 

# Print results

In [39]:
# Display the winning genome.
#print('\nBest genome:\n{!s}'.format(winner))

# Show output of the most fit genome against training data.
winner_network = neat.nn.FeedForwardNetwork.create(winner, config)
visualize.draw_net(config, winner, False)
visualize.plot_stats(stats, ylog = False, view = False)
visualize.plot_species(stats, view = False)

#p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
#p.run(eval_genomes, 10)

# Evaluate from checkpoint & send to OpenAI

In [ ]:
p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-76')
p.run(evaluator.evaluate, 1)
winner = p.best_genome

In [43]:
env_eval = gym.make(game)
monitor_path = '/tmp/' + game + '-eval'
env_eval = wrappers.Monitor(env_eval, monitor_path)
def evaluate_with_video(game, network):
    for i_episode in range(100):
        observation = env_eval.reset()
        total_reward = 0
        t = 0
        while True:
            env_eval.render()
            action = predict_action(observation, network)
            observation, reward, done, info = env_eval.step(action)
            total_reward += reward
            t += 1
            if done:
                print("Episode finished after {0} timesteps reward = {1}".format(t+1, total_reward))
                break

In [ ]:
evaluate_with_video(game, winner_network)

In [ ]:
env_eval.close()

In [ ]:
gym.upload(monitor_path, api_key='sk_ciz2F0csRzCkpESayoRuug')